# Data Preparation

In [2]:
import pandas as pd
metadata = pd.read_csv('gaming_consoles.csv')
metadata.head()

,url,Picture URL,Brand,Product Name,Model,Price in India,Total Ratings,1 Stars,2 Stars,3 Stars,...,USB,Ethernet,Processor,Graphics,Weight,Battery Type,Console Family,Launched,Wi-Fi,HDMI
0,https://gadgets.ndtv.com/nintendo-2ds-xl-12001,https://i.gadgets360cdn.com/products/large/154...,Nintendo,New Nintendo 2DS XL (Black/Turquoise),New 2DS XL,"₹ 27,999",755,9,4,3,...,No,No,804 MHz ARM11 MPCore quad-core,204 MHz DMP PICA200,261g,Li-ion,Nintendo DS,Jun-17,Yes,No
1,https://gadgets.ndtv.com/new-nintendo-3ds-xl-1...,https://i.gadgets360cdn.com/products/large/154...,Nintendo,New Nintendo 3DS XL Monster Hunter (Red),New 3DS XL,"₹ 114,696",237,8,4,4,...,No,No,804 MHz ARM11 MPCore quad-core,204 MHz DMP PICA200,336g,Li-ion,Nintendo DS,Feb-15,Yes,No
2,https://gadgets.ndtv.com/new-nintendo-3ds-xl-1...,https://i.gadgets360cdn.com/products/large/154...,Nintendo,New Nintendo 3DS XL Pikachu Edition (Yellow),New 3DS XL,"₹ 78,847",237,8,4,4,...,No,No,804 MHz ARM11 MPCore quad-core,204 MHz DMP PICA200,336g,Li-ion,Nintendo DS,Feb-15,Yes,No
3,https://gadgets.ndtv.com/new-nintendo-3ds-xl-1...,https://i.gadgets360cdn.com/products/large/154...,Nintendo,New Nintendo 3DS XL Monster Hunter (Blue),New 3DS XL,"₹ 19,990",237,8,4,4,...,No,No,804 MHz ARM11 MPCore quad-core,204 MHz DMP PICA200,336g,Li-ion,Nintendo DS,Feb-15,Yes,No
4,https://gadgets.ndtv.com/nintendo-3ds-11994,https://i.gadgets360cdn.com/products/large/154...,Nintendo,Nintendo 3DS (Black),3DS,"₹ 60,000",386,8,3,6,...,No,No,268MHz Dual-Core ARM11 MPCore,DMP PICA200 GPU,230g,Li-ion,Nintendo DS,Mar-11,Yes,No


In [3]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   url                64 non-null     object
 1   Picture URL        64 non-null     object
 2   Brand              64 non-null     object
 3   Product Name       30 non-null     object
 4   Model              64 non-null     object
 5   Price in India     30 non-null     object
 6   Total Ratings      30 non-null     object
 7   1 Stars            64 non-null     int64 
 8   2 Stars            64 non-null     int64 
 9   3 Stars            64 non-null     int64 
 10  4 Stars            64 non-null     int64 
 11  5 Stars            64 non-null     int64 
 12  Hard Disk          48 non-null     object
 13  Console Type       63 non-null     object
 14  AV Digital Output  47 non-null     object
 15  RAM                61 non-null     object
 16  Input method       54 non-null     object
 17 